In [15]:
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile
import gzip
import shutil
from tqdm import tqdm
import time
from numpy import genfromtxt
import pickle

In [3]:
data = pd.read_csv('5c9f1a05-1dc7-4e83-8dc2-3e5071796c07.FPKM.txt', sep="\t", names = ["gene", 'caseid'])

In [4]:
data

,gene,caseid
0,ENSG00000242268.2,0.033416
1,ENSG00000270112.3,0.002318
2,ENSG00000167578.15,2.042905
3,ENSG00000273842.1,0.000000
4,ENSG00000078237.5,5.219121
...,...,...
60478,ENSG00000105063.17,10.612105
60479,ENSG00000231119.2,0.045903
60480,ENSG00000280861.1,0.000000
60481,ENSG00000123685.7,1.744645


In [5]:
data = data.T

In [6]:
data

,0,1,2,3,4,5,6,7,8,9,...,60473,60474,60475,60476,60477,60478,60479,60480,60481,60482
gene,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
caseid,0.0334161,0.00231829,2.0429,0,5.21912,7.10924,0,0.0103085,145.209,0.230792,...,0,0.00755106,0,0.0268996,0,10.6121,0.0459032,0,1.74464,0


In [7]:
def fix_gene_id(dataframe):
    new_column_names = []
    for item in dataframe.iloc[0]:
        idx = item.find(".")
        new_column_names.append(item[0:idx])
    dataframe.columns = new_column_names
    return dataframe

In [8]:
len(new_column_names)

NameError: name 'new_column_names' is not defined

In [ ]:
data.columns = new_column_names
data = data[1:]

In [ ]:
data

## Ended up downloading useing the cart options in TCGA- everything is already unzipped
### next task - each sheet is in a folder inside a folder - need to open all and load into dataframe 
       ### need to use name of sheet as a column 

In [47]:
#next, walk through folders using os to access each sheet 
rootdir = '/Users/dinakats/Desktop/SPICED/final_project/Data/Kidney/rna_seq/gdc_download_20200321_013801.055810'
def get_list_of_files_to_unpack(rootdir):
    list_of_files_to_unpack =[]

    for directories in os.listdir(rootdir):
        try:
            for filename in os.listdir(rootdir+'/'+ directories):
                if ".gz" in filename: #change to .gz to unpack
                    list_of_files_to_unpack.append(os.path.join(directories,filename))
        except:
            continue
    return list_of_files_to_unpack

In [22]:
list_of_files_to_unpack

['e5e620a1-40eb-40ec-ba43-0910598f7a3c/061086d5-2a01-444b-83ac-f1397b184307.FPKM.txt',
 '393ac068-9b9d-449e-8501-a30344b99923/d29a3b20-5097-4056-8b1d-e5520a655a7c.FPKM.txt',
 '5c938a09-8be2-496e-97db-0f7ebc4349bc/d3f73c0f-d518-4e91-b038-a4360495ee27.FPKM.txt',
 '4a9266c7-1976-4d97-a8cd-ef9c734f6b9b/4020fb48-41a6-40be-bf52-0e45fa2c1799.FPKM.txt',
 'aeca25ec-d1eb-4600-afb3-1d81c9a53639/3baace9e-4b01-4029-a3d3-a93c6040bcee.FPKM.txt',
 'e6abaf87-f3fb-4b42-bbfb-8875d518e5a3/0bab5b45-e0b3-4f59-a8de-5b1065dbe78d.FPKM.txt',
 '88457527-4691-42d0-8f1f-73683fac940e/028acd3f-7d97-4946-87b6-93144650346b.FPKM.txt',
 'ad034541-0ae7-4c94-82de-9649d01989ce/c50805c9-71f7-41bb-97f9-5ef389cbfb62.FPKM.txt',
 '3364c86c-553d-46d9-8bea-b4669ea5d374/7179e3f9-d096-4130-85c6-692493fbb8bc.FPKM.txt',
 'b5b4256a-4282-4ffb-afcc-4efd6ce9a09a/99873dfa-4496-4b83-8d1d-193489fb453d.FPKM.txt',
 'a8d38d6b-2a73-4b3f-878d-4b110fa3e5e6/dfdb90e0-bcf1-4cc6-866e-d81cfc5d9039.FPKM.txt',
 '202d55a0-4ab9-4ad1-88fa-2d79c06bdfe5/2399

In [23]:
len(list_of_files_to_unpack)

1427

In [24]:
len(os.listdir(rootdir))

1430

In [53]:
def unzip_in_folder_bash(rootdir, list_of_files_to_unpack):
    for filename in list_of_files_to_unpack:
        full_path = rootdir+"/"+filename
        ! gunzip "$full_path"

In [54]:
#get list of files to add into dataframe
rootdir = '/Users/dinakats/Desktop/SPICED/final_project/Data/Kidney/rna_seq/gdc_download_20200321_013801.055810'

def get_list_of_files_for_df(rootdir):
    list_of_files_for_df =[]
    directory_names = []

    for directories in os.listdir(rootdir):
        try:
            for filename in os.listdir(rootdir+'/'+ directories):
                if "FPKM" in filename:
                    list_of_files_for_df.append(os.path.join(directories,filename))
                    directory_names.append(directories)
        except:
            continue
    return list_of_files_for_df, directory_names

In [27]:
len(list_of_files_for_df) #perfect

1427

In [44]:
matching = [s for s in list_of_files_for_df if 'd4cd1f75-ab87-4d80-90c5-9504aac851d2' in s]
matching #what i'm looking for in json is file_id

['65509594-c7a3-441c-ae35-026b9f296ef4/d4cd1f75-ab87-4d80-90c5-9504aac851d2.FPKM.txt']

In [ ]:
list_of_files_for_df

In [28]:
len(case_names)

1427

In [ ]:
case_names

In [29]:
case_names[0]

'e5e620a1-40eb-40ec-ba43-0910598f7a3c'

In [ ]:
test1 = rootdir+"/"+list_of_files_to_unpack[0]

In [ ]:
list_of_dfs=[]
for filename in list_of_files_for_df: 
    list_of_dfs.append(pd.read_csv(filename, sep="\t", names = ["gene", 'caseid']))
    

In [ ]:
test1 = pd.read_csv((rootdir+'/'+list_of_files_for_df[0]), sep="\t", names = ["gene", 'test1'], index_col = 'gene')
test2 = pd.read_csv((rootdir+'/'+list_of_files_for_df[1400]), sep="\t", names = ["gene", 'test2'], index_col = 'gene')

In [ ]:
test1

In [ ]:
test2

In [ ]:
test3 = pd.concat([test1,test2], join = "outer", axis = 1)
test3

In [61]:
def dfs_to_merge(list_of_files, case_names, rootdir):
    list_of_dfs = []
    for i, filename in enumerate(list_of_files): 
        list_of_dfs.append(pd.read_csv((rootdir+'/'+filename), sep="\t", names = ["gene", case_names[i]], index_col='gene'))
        print(len(list_of_dfs))
    return list_of_dfs

In [31]:
list_of_dfs = dfs_to_merge(list_of_files_for_df, case_names, rootdir)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [32]:
rna_seq_data = pd.concat(list_of_dfs, join = "outer", axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [33]:
rna_seq_data.head(2)

,e5e620a1-40eb-40ec-ba43-0910598f7a3c,393ac068-9b9d-449e-8501-a30344b99923,5c938a09-8be2-496e-97db-0f7ebc4349bc,4a9266c7-1976-4d97-a8cd-ef9c734f6b9b,aeca25ec-d1eb-4600-afb3-1d81c9a53639,e6abaf87-f3fb-4b42-bbfb-8875d518e5a3,88457527-4691-42d0-8f1f-73683fac940e,ad034541-0ae7-4c94-82de-9649d01989ce,3364c86c-553d-46d9-8bea-b4669ea5d374,b5b4256a-4282-4ffb-afcc-4efd6ce9a09a,...,ec031dae-9e11-4425-9b77-6f5f04a53834,0e903a34-9c87-4444-abfa-fa22882f2cd5,4fd34e9b-70ce-44dd-a1cf-9e699c224479,b60204e2-9014-45f4-93a0-daacfb4c040d,8b8af289-056d-4099-afc5-e4fbbdb08b0b,b91e42dc-1b4c-4df9-b0cb-435dc635c7b9,63150b0a-4b30-41c7-a88e-7ed0ce1b2e0a,38feb06e-2188-4944-ba1a-ded58d6420b1,499222c5-7006-4ca8-a126-f6fa1451f2d5,16b5ab0f-2a0f-4f09-95b2-81e8a18ea8a4
ENSG00000000003.13,15.587179,13.169401,12.928943,31.772067,14.125848,34.5674,8.8396,19.023014,28.4448,10.4930,...,10.9736,11.143487,8.798314,42.626978,10.557532,14.533058,10.069135,54.633514,34.606394,5.654100
ENSG00000000005.5,0.459079,0.066538,0.075844,0.593634,0.093522,0.6107,0.8342,6.957998,0.0571,0.1658,...,0.7627,0.058145,0.603914,3.527074,0.023209,0.230097,0.040131,0.598654,0.139854,8.512874


In [34]:
rna_seq_data.shape

(60483, 1427)

In [35]:
rna_seq_transpose = rna_seq_data.T

In [36]:
rna_seq_transpose.shape

(1427, 60483)

In [37]:
rna_seq_transpose.head(2)

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSGR0000263980.4,ENSGR0000264510.4,ENSGR0000264819.4,ENSGR0000265658.4,ENSGR0000270726.4,ENSGR0000275287.3,ENSGR0000276543.3,ENSGR0000277120.3,ENSGR0000280767.1,ENSGR0000281849.1
e5e620a1-40eb-40ec-ba43-0910598f7a3c,15.587179,0.459079,15.374105,2.015143,0.258277,1.448556,3.946435,24.948585,4.528416,5.492300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
393ac068-9b9d-449e-8501-a30344b99923,13.169401,0.066538,14.600142,1.696474,0.424893,2.677658,1.177146,70.094652,8.879771,5.528283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
 pickle.dump(rna_seq_transpose, open( "rna_seq_transpose.p", "wb" ) )

In [48]:
# need to deal with missing 75 files
root_missing75 = '/Users/dinakats/Desktop/SPICED/final_project/Data/Kidney/rna_seq/gdc_download_20200324_193443.596124'


In [49]:
missing75_unpack = get_list_of_files_to_unpack(root_missing75)

In [50]:
len(missing75_unpack)

75

In [51]:
missing75_unpack

['93d8dcc3-0f02-4bc0-bbbb-f42882bd0908/fb783ec9-4ca1-4354-bb51-29b5bbb909b3.FPKM.txt.gz',
 '76ddc938-a9df-452a-838b-b813c586b6b6/384235f4-4697-4069-a759-0481999fd290.FPKM.txt.gz',
 '90826ed6-3578-4cc3-a3df-0de36d5d03f6/9b9e9ba2-f934-40a2-a312-d876707c37d2.FPKM.txt.gz',
 '989dc29c-b27f-42a0-880b-07a722d5ce39/20144c76-d6ee-4589-aac6-0fe8d4174197.FPKM.txt.gz',
 '26e13369-76bb-4aa1-b4be-102bbc697d1b/12a2d93e-8567-4a55-a3f3-4c94c51ffb6a.FPKM.txt.gz',
 '889b48f7-c8cf-4eff-8d08-93ab0def745a/61cce01f-7db5-4a8e-806a-94b1dda47997.FPKM.txt.gz',
 '68e4cb24-f646-45c0-a623-55980a170892/d6b51a36-7ce9-4044-809b-e48e9213abd8.FPKM.txt.gz',
 'b603fb1e-49a6-4663-8881-1e8a2262f4e6/138485c1-4e31-4d40-a1c0-af9448f4b650.FPKM.txt.gz',
 '736590b9-f46b-4c75-83c7-b70cf187b4e5/557ca9d1-a5be-4252-9abc-0ae3142043e9.FPKM.txt.gz',
 '725925f1-b9b3-4f27-a5d3-675f2ebe7dc3/9235a5b1-2a3b-448b-91c5-59c92180c66f.FPKM.txt.gz',
 'b68d2a82-7558-4cf6-893d-9afd14409555/2a29763d-bf90-4489-b26f-1f05375aa1ff.FPKM.txt.gz',
 'e285a151

In [52]:
#run unpacking
for filename in missing75_unpack:
    full_path = root_missing75+"/"+filename
    ! gunzip "$full_path"
#looks like it worked

In [55]:
missing75_for_df, directory75_names = get_list_of_files_for_df(root_missing75)

In [59]:
missing75_for_df

['93d8dcc3-0f02-4bc0-bbbb-f42882bd0908/fb783ec9-4ca1-4354-bb51-29b5bbb909b3.FPKM.txt',
 '76ddc938-a9df-452a-838b-b813c586b6b6/384235f4-4697-4069-a759-0481999fd290.FPKM.txt',
 '90826ed6-3578-4cc3-a3df-0de36d5d03f6/9b9e9ba2-f934-40a2-a312-d876707c37d2.FPKM.txt',
 '989dc29c-b27f-42a0-880b-07a722d5ce39/20144c76-d6ee-4589-aac6-0fe8d4174197.FPKM.txt',
 '26e13369-76bb-4aa1-b4be-102bbc697d1b/12a2d93e-8567-4a55-a3f3-4c94c51ffb6a.FPKM.txt',
 '889b48f7-c8cf-4eff-8d08-93ab0def745a/61cce01f-7db5-4a8e-806a-94b1dda47997.FPKM.txt',
 '68e4cb24-f646-45c0-a623-55980a170892/d6b51a36-7ce9-4044-809b-e48e9213abd8.FPKM.txt',
 'b603fb1e-49a6-4663-8881-1e8a2262f4e6/138485c1-4e31-4d40-a1c0-af9448f4b650.FPKM.txt',
 '736590b9-f46b-4c75-83c7-b70cf187b4e5/557ca9d1-a5be-4252-9abc-0ae3142043e9.FPKM.txt',
 '725925f1-b9b3-4f27-a5d3-675f2ebe7dc3/9235a5b1-2a3b-448b-91c5-59c92180c66f.FPKM.txt',
 'b68d2a82-7558-4cf6-893d-9afd14409555/2a29763d-bf90-4489-b26f-1f05375aa1ff.FPKM.txt',
 'e285a151-7a28-4ae4-a038-e3573d279058/2540

In [62]:
#make a df 
missing75_df_list = dfs_to_merge(missing75_for_df, directory75_names, root_missing75)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75


In [63]:
rna_seq_missing75 = pd.concat(missing75_df_list, join = "outer", axis = 1)

In [65]:
rna_seq_missing75 = rna_seq_missing75.T

In [66]:
rna_seq_missing75

gene,ENSG00000242268.2,ENSG00000270112.3,ENSG00000167578.15,ENSG00000273842.1,ENSG00000078237.5,ENSG00000146083.10,ENSG00000225275.4,ENSG00000158486.12,ENSG00000198242.12,ENSG00000259883.1,...,ENSG00000238244.3,ENSG00000186115.11,ENSG00000216352.1,ENSG00000267117.1,ENSG00000273233.1,ENSG00000105063.17,ENSG00000231119.2,ENSG00000280861.1,ENSG00000123685.7,ENSG00000181518.3
93d8dcc3-0f02-4bc0-bbbb-f42882bd0908,0.023698,0.003288,2.149666,0.258854,3.389593,5.223044,0.000000,0.004499,125.321490,0.021823,...,0.0,0.000000,0.000000,0.076306,0.000000,14.921367,0.260428,0.0,5.873116,0.0
76ddc938-a9df-452a-838b-b813c586b6b6,0.000000,0.004399,2.835796,0.000000,2.993275,6.673202,0.000000,0.012037,156.782804,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,13.612161,0.012443,0.0,1.402154,0.0
90826ed6-3578-4cc3-a3df-0de36d5d03f6,0.587108,0.015666,1.242045,0.000000,3.288186,11.690172,0.000000,0.143607,55.864161,0.062384,...,0.0,0.500061,0.000000,0.218130,0.112905,18.489928,0.017725,0.0,4.745354,0.0
989dc29c-b27f-42a0-880b-07a722d5ce39,0.000000,0.019033,1.469383,0.000000,1.945866,4.047016,0.000000,0.007812,148.919693,0.000000,...,0.0,0.012398,0.000000,0.000000,0.000000,7.349737,0.893683,0.0,2.204107,0.0
26e13369-76bb-4aa1-b4be-102bbc697d1b,0.171723,0.000000,1.681151,0.000000,2.725282,7.967171,0.000000,0.006986,93.539464,0.090364,...,0.0,0.027717,0.000000,0.039496,0.081773,11.337453,0.057770,0.0,2.234179,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b9d2df0d-8f37-417e-b18c-797c0764a60a,0.000000,0.000000,1.824378,0.000000,3.646083,3.991230,0.000000,0.012169,81.975005,0.021465,...,0.0,0.000000,0.000000,0.037528,0.000000,7.411296,0.384239,0.0,2.440061,0.0
85ce8fa6-1bf6-4470-aabf-d5c0c43ca794,0.000000,0.014443,2.822040,0.000000,2.672931,4.587502,0.000000,0.009880,43.573546,0.255616,...,0.0,0.000000,0.000000,0.223446,0.000000,12.861979,0.313214,0.0,2.524336,0.0
9a687f17-aa26-445a-9acc-9a1b492c3c1a,0.041985,0.002913,3.452633,0.000000,2.080632,5.848616,0.000000,0.019926,146.449539,0.096658,...,0.0,0.000000,0.000000,0.000000,0.000000,17.902303,0.049435,0.0,1.444864,0.0
46fa3b22-c39a-4f50-a0f4-1718092fc75e,0.000000,0.000000,2.107524,0.000000,1.650426,5.541257,0.000000,0.010875,112.545672,0.023446,...,0.0,0.000000,0.000000,0.000000,0.084868,9.254038,0.539611,0.0,1.925632,0.0


In [67]:
 pickle.dump(rna_seq_missing75, open( "rna_seq_missing75.p", "wb" ) )